In [ ]:
#### Start in KI_Compartment root folder
# You must have SymITS2 and all its dependencies installed and working from your home folder (i.e. at ~/SymITS2)
# You must have a working version of QIIME, with UCLUST and USEARCH61 installed.
# You must have the nucleotide NCBI blast database locally on your machine, and a working version of blast
    # You might want to put this on a hard drive, as it is currently ~94GB of data (as of August 1, 2017)
    # Download Blastdb from ftp://ftp.ncbi.nlm.nih.gov/blast/documents/blastdb.html, dataset: nt.*tar.gz.
    # Then make sure to unzip all folders and place in one folder called "blastdb"
    # Mine is at "/media/danielle/CoralDrive/blastdb"
    # For getting blast installed on Linux Ubuntu, this website was helpful: http://www.blopig.com/blog/2014/04/quick-standalone-blast-setup-for-ubuntu-linux/
# You must have an updated version of R, I used R 3.4.1 (2017-06-30, Single Candle)
    # Required R packages are specified in SymITS2

# Prep raw data
#chmod u+x data/Bioinf/extract_unzip_prep_rawdata.sh # Make sure script can run
#data/Bioinf/extract_unzip_prep_rawdata.sh # This does the following:
    # Unzips all original files
    # Makes configs for Bokulich qc
    # Runs Bokulich qc
    # Makes configs for Illumina paired end merging
    # Merges Illumina pairs, includes Q30 check
    # Filters merged reads
    
# Or run each command from extract_unzip_prep_rawdata individually...
find . -type f -exec gunzip {} + # Unzip these files
# They are going to get really big! And this may take quite some time!

python data/Bioinf/make_configs.py # Make the Illumina-utils config files for the Bokulich QC method

python data/Bioinf/boku_qc.py # Run Bokulich QC method on all applicable files
chmod u+x data/Bioinf/boku_qc_KI_Compartment.sh # Give permissions so .sh file will run

source ~/virtual-envs/illumina-utils-v2.0.0/bin/activate # Activate Illumina-utils
data/Bioinf/boku_qc_KI_Compartment.sh # Run .sh file that was just created 

python data/Bioinf/make_merge_configs.py # Make the Illumina-utils config files for merging pairs

python data/Bioinf/iu_merge_pairs.py # Merge pairs using Illumina-utils
chmod u+x data/Bioinf/iu-merge-pairs_KI_Compartment.sh # Give permissions so .sh file will run
data/Bioinf/iu-merge-pairs_KI_Compartment.sh # Run .sh file that was just created 
# find . -maxdepth 1 -name # To check how far along the merging is (because it takes a long time to do hundreds of samples)

python data/Bioinf/iu_filter_merged_reads.py # Filter merged reads (MAX-MISMATCH=3) using Illumina-utils
chmod u+x data/Bioinf/iu-filter-merged-reads_KI_Compartment.sh # Give permissions so .sh file will run
data/Bioinf/iu-filter-merged-reads_KI_Compartment.sh # Run .sh file that was just created 

python data/Bioinf/rename.py # Rename merged files for downstream processing

In [ ]:
# Now, because I named these samples unneccesarily differently when I sent them in for sequencing, let's fix that before moving downstream...

rename 's/DClaar_2014_S_\.*/KI14SSYM0/' *.fasta
rename 's/DClaar_2014_W_\.*/KI14WSYM0/' *.fasta
rename 's/DClaar_2015a_S_\.*/KI15aSSYM0/' *.fasta
rename 's/DClaar_2015a_W_\.*/KI15aWSYM0/' *.fasta
rename 's/DClaar_2015b_W_\.*/KI15bWSYM0/' *.fasta
rename 's/DClaar_2015b_S_\.*/KI15bSSYM0/' *.fasta


In [5]:
# Next, start using SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
~/SymITS2/qc_trim_reads.sh # This does the following:
    # Adds QIIME labels
    # Identifies chimeric sequences
    # Filters out chimeric sequences
    # Runs cutadapt 4 times
    # Removes intermediate files from cutadapt process

This is cutadapt 1.9.1 with Python 2.7.12
Command line parameters: -g GTGAATTGCAGAACTCCGTG -e 0.15 data/fasta/combined_seqs_chimera_filtered.fasta --trimmed-only -o data/fasta/trimF.fasta
Trimming 1 adapter with at most 15.0% errors in single-end mode ...
Finished in 144.72 s (20 us/read; 3.05 M reads/minute).

=== Summary ===

Total reads processed:               7,353,302
Reads with adapters:                 7,345,426 (99.9%)
Reads written (passing filters):     7,345,426 (99.9%)

Total basepairs processed: 2,556,539,985 bp
Total written (filtered):  2,407,039,381 bp (94.2%)

=== Adapter 1 ===

Sequence: GTGAATTGCAGAACTCCGTG; Type: regular 5'; Length: 20; Trimmed: 7345426 times.

No. of allowed errors:
0-5 bp: 0; 6-12 bp: 1; 13-19 bp: 2; 20 bp: 3

Overview of removed sequences
length	count	expect	max.err	error counts
3	12561	114895.3	0	12561
7	1	448.8	1	0 1
8	10	112.2	1	0 10
9	205	28.1	1	143 62
10	26	7.0	1	21 5
11	30	1.8	1	9 21
12	111	0.4	1	61 50
13	271	0.1	1	165 105 1
14	310	0.0	2	5

In [6]:
count_seqs.py -i data/fasta/combined_seqs_trimmed.fasta # Count how many seqs there are after quality filtering


7332473  : data/fasta/combined_seqs_trimmed.fasta (Sequence lengths (mean +/- std): 306.5629 +/- 24.1155)
7332473  : Total


In [7]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Cluster at 97% within samples (each sample is clustered independently)
~/SymITS2/otus_97_bysample.sh data/fasta/combined_seqs_trimmed.fasta data/Bioinf/clust
# Arguments are 1) Combined, trimmed sequences (fasta file) and 2)Output directory
# Creates data/Bioinf/clust/all_rep_set_rep_set.fasta for downstream use

When using programs that use GNU Parallel to process data for publication please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; and it won't cost you a cent.
Or you can get GNU Parallel without this requirement by paying 10000 EUR.

To silence this citation notice run 'parallel --bibtex' once or use '--no-notice'.

When using programs that use GNU Parallel to process data for publication please cite:

  O. Tange (2011): GNU Parallel - The Command-Line Power Tool,
  ;login: The USENIX Magazine, February 2011:42-47.

This helps funding further development; and it won't cost you a cent.
Or you can get GNU Parallel without this requirement by paying 10000 EUR.

To silence this citation notice run 'parallel --bibtex' once or use '--no-notice'.



In [8]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Use global alignment to identify sequences
R --vanilla < ~/SymITS2/run_nw.R --args data/Bioinf/clust/all_rep_set_rep_set.fasta data/ITS2db_trimmed_derep.fasta
# Arguments are 1) query sequences - rep set fasta file, and 2) reference database sequences in .fasta format
# Creates data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv for downstream use


R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # Get command line arguments
> args = commandArgs(trailingOnly=TRUE)
> # If two arguments not provided, return an error
> if (length(args) < 2) {
+   stop("must specify query sequences (.fasta) and reference sequences (.fasta)", call.=FALSE)
+ }
> 
> # Import query sequences
> library(Biostrings)
Loading req

In [9]:
# Now extract metadata from full metadata spreadsheet
# Using mapping_file.txt
R --vanilla < data/Coralphoto__Metadata/process_coral_metadata.R 



R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> library(plyr)
> 
> rm(list=ls())
> 
> meta <- read.csv("data/Coralphoto__Metadata/KI_Coralphoto_Metadata_Jan_to_Apr_2017_23March.csv",header=T)
> meta$Year_Pre_Post <- paste(meta$field_season,meta$before.after, sep="")
> meta$ref <- paste(meta$Year_Pre_Post,".tag",meta$coral_tag, sep="")
> meta.forcat <- met

43   KI14FSYM087  KI14FSYM087.fasta        87      coral       KI2014
44   KI14FSYM088  KI14FSYM088.fasta        88      coral       KI2014
45   KI14FSYM090  KI14FSYM090.fasta    90_690      coral       KI2014
46   KI14FSYM092  KI14FSYM092.fasta    92_860      coral       KI2014
47   KI14FSYM094  KI14FSYM094.fasta        94      coral       KI2014
48   KI14FSYM097  KI14FSYM097.fasta        97      coral       KI2014
49   KI14FSYM098  KI14FSYM098.fasta        98      coral       KI2014
50   KI14FSYM102  KI14FSYM102.fasta       102      coral       KI2014
51   KI14FSYM105  KI14FSYM105.fasta       105      coral       KI2014
52   KI14FSYM106  KI14FSYM106.fasta       106      coral       KI2014
53   KI14FSYM107  KI14FSYM107.fasta       107      coral       KI2014
54   KI14FSYM109  KI14FSYM109.fasta       109      coral       KI2014
55   KI14FSYM126  KI14FSYM126.fasta       126      coral       KI2014
56   KI14FSYM129  KI14FSYM129.fasta       129      coral       KI2014
57   KI14FSYM131  KI

161 KI15aFSYM110 KI15aFSYM110.fasta       109      coral KI2015a_Post
162 KI15aFSYM111 KI15aFSYM111.fasta        50      coral KI2015a_Post
163 KI15aFSYM114 KI15aFSYM114.fasta        60      coral KI2015a_Post
164 KI15aFSYM117 KI15aFSYM117.fasta        63      coral KI2015a_Post
165 KI15aFSYM118 KI15aFSYM118.fasta    69_875      coral KI2015a_Post
166 KI15aFSYM120 KI15aFSYM120.fasta        76      coral KI2015a_Post
167 KI15aFSYM121 KI15aFSYM121.fasta        84      coral KI2015a_Post
168 KI15aFSYM122 KI15aFSYM122.fasta      126b      coral KI2015a_Post
169 KI15aFSYM126 KI15aFSYM126.fasta       131      coral KI2015a_Post
170 KI15aFSYM127 KI15aFSYM127.fasta       129      coral KI2015a_Post
171 KI15aFSYM128 KI15aFSYM128.fasta       132      coral KI2015a_Post
172 KI15aFSYM129 KI15aFSYM129.fasta       133      coral KI2015a_Post
173 KI15aFSYM130 KI15aFSYM130.fasta       134      coral KI2015a_Post
174 KI15aFSYM131 KI15aFSYM131.fasta       135      coral KI2015a_Post
175 KI15aFSYM132 KI1

279 KI15bFSYM063 KI15bFSYM063.fasta       434      coral      KI2015b
280 KI15bFSYM064 KI15bFSYM064.fasta       436      coral      KI2015b
281 KI15bFSYM066 KI15bFSYM066.fasta       437      coral      KI2015b
282 KI15bFSYM070 KI15bFSYM070.fasta       443      coral      KI2015b
283 KI15bFSYM071 KI15bFSYM071.fasta       440      coral      KI2015b
284 KI15bFSYM073 KI15bFSYM073.fasta       446      coral      KI2015b
285 KI15bFSYM074 KI15bFSYM074.fasta       441      coral      KI2015b
286 KI15bFSYM128 KI15bFSYM128.fasta       682      coral      KI2015b
287 KI15bFSYM129 KI15bFSYM129.fasta       133      coral      KI2015b
288 KI15bFSYM130 KI15bFSYM130.fasta       147      coral      KI2015b
289 KI15bFSYM133 KI15bFSYM133.fasta   445_683      coral      KI2015b
290 KI15bFSYM134 KI15bFSYM134.fasta       476      coral      KI2015b
291 KI15bFSYM135 KI15bFSYM135.fasta       474      coral      KI2015b
292 KI15bFSYM137 KI15bFSYM137.fasta       477      coral      KI2015b
293 KI15bFSYM141 KI1

15          11              sediment   35   <NA>          2014
16          12        Porites_lobata   35   gone          2014
17          12              sediment   35   <NA>          2014
18          13   Pocillopora_eydouxi   35   dead          2014
19          13              sediment   35   <NA>          2014
20          14              sediment   35   <NA>          2014
21          15        Porites_lobata   35  alive          2014
22          15              sediment   30   <NA>          2014
23          16              sediment   30   <NA>          2014
24          17              sediment   30   <NA>          2014
25          18              sediment   30   <NA>          2014
26          19              sediment   30   <NA>          2014
27          20              sediment   30   <NA>          2014
28          41     Montipora_foliosa   27     UK          2014
29          43   Pocillopora_eydouxi   27     UK          2014
30          44     Montipora_foliosa   27   gone       

146         87        Porites_lobata   35     UK  2015Jan_Post
147         89   Pocillopora_eydouxi   35   dead  2015Jan_Post
148         90     Montipora_foliosa   35   dead  2015Jan_Post
149         91        Porites_lobata   35   gone  2015Jan_Post
150         93        Porites_lobata   35  alive  2015Jan_Post
151         94   Pocillopora_eydouxi   35     UK  2015Jan_Post
152         95   Pocillopora_eydouxi   35   dead  2015Jan_Post
153         99   Pocillopora_eydouxi   35   gone  2015Jan_Post
154        100        Porites_lobata   35     UK  2015Jan_Post
155        101     Montipora_foliosa   35   dead  2015Jan_Post
156        102     Montipora_foliosa   35   dead  2015Jan_Post
157        106     Montipora_foliosa   35   dead  2015Jan_Post
158        107   Pocillopora_eydouxi   35   dead  2015Jan_Post
159        108     Montipora_foliosa   35   dead  2015Jan_Post
160        109        Porites_lobata   35     UK  2015Jan_Post
161        110   Pocillopora_eydouxi   35   dead  2015J

277         60   Pocillopora_eydouxi   30   dead       2015May
278         62        Porites_lobata   30     UK       2015May
279         63        Porites_lobata   30   gone       2015May
280         64   Pocillopora_eydouxi   30     UK       2015May
281         66     Montipora_foliosa   30   dead       2015May
282         70     Montipora_foliosa   30   dead       2015May
283         71   Pocillopora_eydouxi   30   dead       2015May
284         73     Montipora_foliosa   30     UK       2015May
285         74        Porites_lobata   30  alive       2015May
286        128        Porites_lobata   30   gone       2015May
287        129     Montipora_foliosa   30   gone       2015May
288        130     Montipora_foliosa   30     UK       2015May
289        133        Porites_lobata   30   gone       2015May
290        134        Porites_lobata   27   gone       2015May
291        135        Porites_lobata   27   gone       2015May
292        137   Pocillopora_eydouxi   27   dead       

44                    KI2014.tag88
45                KI2014.tag90_690
46                KI2014.tag92_860
47                    KI2014.tag94
48                    KI2014.tag97
49                    KI2014.tag98
50                   KI2014.tag102
51                   KI2014.tag105
52                   KI2014.tag106
53                   KI2014.tag107
54                   KI2014.tag109
55                   KI2014.tag126
56                   KI2014.tag129
57                   KI2014.tag131
58                   KI2014.tag132
59                   KI2014.tag133
60                   KI2014.tag134
61                   KI2014.tag135
62                   KI2014.tag136
63                   KI2014.tag137
64                   KI2014.tag140
65                   KI2014.tag142
66                   KI2014.tag143
67                   KI2014.tag144
68                   KI2014.tag147
69                   KI2014.tag149
70                   KI2014.tag151
71                   KI2014.tag265
72                  

279                 KI2015b.tag434
280                 KI2015b.tag436
281                 KI2015b.tag437
282                 KI2015b.tag443
283                 KI2015b.tag440
284                 KI2015b.tag446
285                 KI2015b.tag441
286                 KI2015b.tag682
287                 KI2015b.tag133
288                 KI2015b.tag147
289             KI2015b.tag445_683
290                 KI2015b.tag476
291                 KI2015b.tag474
292                 KI2015b.tag477
293             KI2015b.tag451_871
294                 KI2015b.tag456
295                 KI2015b.tag469
296                  KI2015b.tag44
297                  KI2015b.tag46
298                 KI2015b.tag684
299                 KI2015b.tag688
300              KI2015b.tag90_690
301                 KI2015b.tag109
302             KI2015b.tag114_692
303             KI2015b.tag121_693
304                KI2015b.tag122b
305                 KI2015b.tag123
306             KI2015b.tag124_694
307                 

In [11]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Build a phyloseq object
R --vanilla < ~/SymITS2/build_phyloseq.R --args data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv data/Coralphoto__Metadata/KI_Compartment_metadata.tsv data/Bioinf/clust/97_otus_bysample.tsv data/ITS2db_trimmed_notuniques_otus.txt analyses/KI_Compartment.RData
# Arguments are 1) nw_tophits - taxonomic assignment output from run_nw.R, 2) sample metadata in tsv format, 3) OTU table, 4) duplicate reference taxa names, 5) output filename
# Creates analyses/KI_Compartment.RData


R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # Script to build phyloseq object
> library(stringr)
> library(phyloseq)
> library(reshape2)
> 
> args= c("data/Bioinf/clust/all_rep_set_rep_set_nw_tophits.tsv", 
+         "data/Coralphoto__Metadata/KI_Compartment_metadata.tsv", 
+         "data/Bioinf/clust/97_otus_bysample.tsv", 
+         "data/ITS2db_tr

In [12]:
# From SymITS2 by Ross Cunning https://github.com/jrcunning/SymITS2
# Filter out sequences which do not blast to Symbiodnium
R --vanilla < ~/SymITS2/filter_notsym.R --args analyses/KI_Compartment.RData data/Bioinf/clust/all_rep_set_rep_set.fasta analyses/KI_Compartment.RData /media/danielle/CoralDrive/blastdb
# Output file is KI_Compartment.RData for downstream analysis (next step = analyses/KI_Compartment_filter_samples.R)


R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # FILTER OUT OTUS THAT ARE NOT SYMBIODINIUM
> library(stringr)
> library(Biostrings)
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterE

In [7]:
R --vanilla < data/Bioinf/export_physeq_objects.R



R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # Exporting phyloseq objects
> 
> load("analyses/KI_Compartment.RData")
> 
> # Write tax table for phylogenetically-informed diversity
> write.table(data.frame(phyloseq::tax_table(phy.f)), "data/tax_table.txt", row.names=T, quote=F)
> # Write otu table for phylogenetically-informed diversity
> write.table(da

In [5]:
# Build the phylogenetic tree to add to phyloseq object

#Originally from Moorea_Sym pipeline
#data/Bioinf/clust/all_rep_set_rep_set.fasta = seqs
#data/tax_table.txt = ISDs
#column 9 is clade ID
#awk '{print $1}' data/tax_table.txt = denovo name
#awk '{print $8}' data/tax_table.txt = clade name

#subset ID lists by clade
awk '$9 ~ /^A/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/A_tree_seq.ids
awk '$9 ~ /^B/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/B_tree_seq.ids
awk '$9 ~ /^C/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/C_tree_seq.ids
awk '$9 ~ /^D/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/D_tree_seq.ids
awk '$9 ~ /^E/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/E_tree_seq.ids
awk '$9 ~ /^F/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/F_tree_seq.ids
awk '$9 ~ /^G/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/G_tree_seq.ids
awk '$9 ~ /^H/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/H_tree_seq.ids
awk '$9 ~ /^I/{ print $1; }' data/tax_table.txt > data/Bioinf/tree/I_tree_seq.ids

In [6]:
#filter seqs by id list
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/A_tree_seq.ids -o data/Bioinf/tree/A_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/B_tree_seq.ids -o data/Bioinf/tree/B_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/C_tree_seq.ids -o data/Bioinf/tree/C_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/D_tree_seq.ids -o data/Bioinf/tree/D_tree_seqs.fasta
#filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/E_tree_seq.ids -o data/Bioinf/tree/E_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/F_tree_seq.ids -o data/Bioinf/tree/F_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/G_tree_seq.ids -o data/Bioinf/tree/G_tree_seqs.fasta
#filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/H_tree_seq.ids -o data/Bioinf/tree/H_tree_seqs.fasta
filter_fasta.py -f data/Bioinf/clust/all_rep_set_rep_set.fasta -s data/Bioinf/tree/I_tree_seq.ids -o data/Bioinf/tree/I_tree_seqs.fasta

#check seq number
grep -c ">" data/Bioinf/tree/A_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/B_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/C_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/D_tree_seqs.fasta
#grep -c ">" data/Bioinf/tree/E_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/F_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/G_tree_seqs.fasta
#grep -c ">" data/Bioinf/tree/H_tree_seqs.fasta
grep -c ">" data/Bioinf/tree/I_tree_seqs.fasta

#align fasta files for each clade
align_seqs.py -i data/Bioinf/tree/A_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/B_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/C_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/D_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
#align_seqs.py -i data/Bioinf/tree/E_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/F_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/G_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
#align_seqs.py -i data/Bioinf/tree/H_tree_seqs.fasta -m muscle -o data/Bioinf/tree/
align_seqs.py -i data/Bioinf/tree/I_tree_seqs.fasta -m muscle -o data/Bioinf/tree/

#remove extra header info from alignments
sed 's/ .*//' data/Bioinf/tree/A_tree_seqs_aligned.fasta > data/Bioinf/tree/A_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/B_tree_seqs_aligned.fasta > data/Bioinf/tree/B_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/C_tree_seqs_aligned.fasta > data/Bioinf/tree/C_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/D_tree_seqs_aligned.fasta > data/Bioinf/tree/D_tree_seqs_aligned_clean.fasta
#sed 's/ .*//' data/Bioinf/tree/E_tree_seqs_aligned.fasta > data/Bioinf/tree/E_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/F_tree_seqs_aligned.fasta > data/Bioinf/tree/F_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/G_tree_seqs_aligned.fasta > data/Bioinf/tree/G_tree_seqs_aligned_clean.fasta
#sed 's/ .*//' data/Bioinf/tree/H_tree_seqs_aligned.fasta > data/Bioinf/tree/H_tree_seqs_aligned_clean.fasta
sed 's/ .*//' data/Bioinf/tree/I_tree_seqs_aligned.fasta > data/Bioinf/tree/I_tree_seqs_aligned_clean.fasta

56
1
350
51
66
26
7


In [1]:
# Format and filter the RData file
R --vanilla < analyses/KI_Compartment_filter_samples.R
# Input file is analyses/KI_Compartment.RData
# Must run build_phy_tree first for this to work properly
# Output file is: data/KI_Compartment_f_coral_grouped.RData


R version 3.4.1 (2017-06-30) -- "Single Candle"
Copyright (C) 2017 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> # Import Libraries
> library(stringr)
> library(reshape2)
> library(phyloseq)
> library(seqinr)
> library(phangorn) 
Loading required package: ape

Attaching package: ‘ape’

The following objects are masked from ‘package:seqinr’:

    as.alignment, consensus

> library(caroline)
> 
> # Clear workspace
> rm(l

> C_G <- matrix(0.187, ncol=ncol(C.dis), nrow=nrow(G.dis), dimnames=list(rownames(G.dis), colnames(C.dis)))
> C_I <- matrix(0.137, ncol=ncol(C.dis), nrow=nrow(I.dis), dimnames=list(rownames(I.dis), colnames(C.dis)))
> D_F <- matrix(0.1691, ncol=ncol(D.dis), nrow=nrow(F.dis), dimnames=list(rownames(F.dis), colnames(D.dis)))
> D_G <- matrix(0.1795, ncol=ncol(D.dis), nrow=nrow(G.dis), dimnames=list(rownames(G.dis), colnames(D.dis)))
> D_I <- matrix(0.169125, ncol=ncol(D.dis), nrow=nrow(I.dis), dimnames=list(rownames(I.dis), colnames(D.dis)))
> F_G <- matrix(0.2072, ncol=ncol(F.dis), nrow=nrow(G.dis), dimnames=list(rownames(G.dis), colnames(F.dis)))
> F_I <- matrix(0.14925, ncol=ncol(F.dis), nrow=nrow(I.dis), dimnames=list(rownames(I.dis), colnames(F.dis)))
> G_I <- matrix(0.194, ncol=ncol(G.dis), nrow=nrow(I.dis), dimnames=list(rownames(I.dis), colnames(G.dis)))
> 
> #build ACDG matrix
> col1 <- rbind(A.dis, A_B, A_C, A_D, A_F, A_G, A_I)
> col2 <- rbind(matrix(NA, nrow=nrow(A.dis), ncol=n